In [2]:
# use magic command to auto-reload
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import igraph as ig
import json
from dotenv import load_dotenv
import requests

sys.path.insert(0, os.path.join(os.getcwd(), '..'))  # parent folder of src
from src.simulation import run_simulation
from src.save import save_simulation_results, show_discussion_from_saved, load_replica
from src.visualization import show_interactive_conversation_tree_with_agent_selection
import dotenv
import os
import asyncio
import aiohttp

# Load environment variables
env = dotenv.dotenv_values(".env")

In [3]:
import requests
import dotenv
from typing import List, Dict, Any

env = dotenv.dotenv_values(".env")

class Agent:
    def __init__(self, name: str, persona: str):
        self.name = name
        self.persona = persona
        self.posts = []

class Conversation:
    def __init__(self, agents: List[Agent], topic: str, prompt_format: str = "threaded"):
        self.agents = {agent.name: agent for agent in agents}
        self.topic = topic
        self.prompt_format = prompt_format
        self.timeline = []  # [(agent_name, message, timestamp)]
        
    def generate_prompt(self, agent_name: str, context_length: int = 5) -> str:
        agent = self.agents[agent_name]
        recent_timeline = self.timeline[-context_length:] if self.timeline else []
        
        if self.prompt_format == "flat":
            return self._flat_prompt(agent, recent_timeline)
        elif self.prompt_format == "threaded":
            return self._threaded_prompt(agent, recent_timeline)
        elif self.prompt_format == "context_window":
            return self._context_window_prompt(agent, recent_timeline)
        elif self.prompt_format == "roleplay":
            return self._roleplay_prompt(agent, recent_timeline)
    
    def _flat_prompt(self, agent: Agent, timeline: List) -> str:
        context = "\n".join([f"{name}: \"{msg}\"" for name, msg, _ in timeline])
        return f"""{agent.persona}

Recent posts:
{context}

Write your next social media post responding to this conversation. Keep it under 250 characters."""

    def _threaded_prompt(self, agent: Agent, timeline: List) -> str:
        if not timeline:
            context = "[No previous messages]"
        else:
            context = "Conversation thread:\n"
            for name, msg, _ in timeline:
                context += f"{name}: \"{msg}\"\n"
            context += "└─ [Your turn to respond]"
        
        return f"""{agent.persona}

{context}

Write your response to this thread. You can reply to any person or the general topic. Keep it under 250 characters."""

    def _context_window_prompt(self, agent: Agent, timeline: List) -> str:
        other_agents = [name for name in self.agents.keys() if name != agent.name]
        context_header = f"[CONTEXT: You're discussing {self.topic} with {', '.join(other_agents)}]"
        
        discussion = ""
        for name, msg, _ in timeline:
            discussion += f"{name} said: \"{msg}\"\n"
        
        return f"""{agent.persona}

{context_header}

The discussion so far:
{discussion}
Now write your response to this {self.topic} discussion. Keep it under 250 characters."""

    def _roleplay_prompt(self, agent: Agent, timeline: List) -> str:
        scene = f"SCENE: Group discussion about {self.topic}\n\n"
        for name, msg, _ in timeline:
            scene += f"{name.upper()}: \"{msg}\"\n"
        scene += f"{agent.name.upper()} (you): [Write your response here]"
        
        return f"""{agent.persona}

{scene}

Stay in character as {agent.name}. Keep your response under 250 characters."""

def send_prompt(prompt: str) -> str:
    headers = {"Authorization": f"Bearer {env['HF_TOKEN']}"}
    payload = {
        "messages": [{"role": "user", "content": prompt}],
        "model": env["MODEL"],
        "max_tokens": 128,
        "temperature": 0.7
    }
    
    response = requests.post(env["API_URL"], headers=headers, json=payload)
    data = response.json()
    
    if "choices" in data:
        return data["choices"][0]["message"]["content"].strip().strip('"')
    else:
        raise Exception(f"API Error: {data}")

def probe_opinion(agent_name: str, conversation: Conversation, topic: str) -> int:
    """Probe agent's stance based on their actual conversation behavior (1-5 scale)"""
    agent = conversation.agents[agent_name]
    
    # Get agent's own posts from the conversation
    agent_posts = []
    for name, msg, _ in conversation.timeline:
        if name == agent_name:
            agent_posts.append(msg)
    
    if not agent_posts:
        # No posts yet, probe based on persona
        probe_prompt = f"""{agent.persona}

On a scale of 1-5, how much do you support {topic}?
1 = Strongly oppose
2 = Somewhat oppose  
3 = Neutral
4 = Somewhat support
5 = Strongly support

Answer with only the number."""
    else:
        # Probe based on actual posts
        posts_text = "\n".join([f"- \"{post}\"" for post in agent_posts])
        probe_prompt = f"""Based on your recent posts in the {topic} discussion:

{posts_text}

What stance have you been taking in this conversation?
1 = Strongly opposing {topic}
2 = Somewhat opposing  
3 = Neutral/mixed
4 = Somewhat supporting
5 = Strongly supporting

Answer with only the number."""
    
    response = send_prompt(probe_prompt)
    return int(response.strip())

def run_conversation_round(conversation: Conversation, context_length: int = 5, 
                          track_opinions: bool = False) -> Dict[str, Any]:
    """Run one round where each agent posts once"""
    round_data = {"responses": {}, "opinions": {}}
    
    # Track opinions before round if requested
    if track_opinions:
        for agent_name in conversation.agents.keys():
            opinion = probe_opinion(agent_name, conversation, conversation.topic)
            round_data["opinions"][agent_name] = opinion
    
    # Generate responses
    for agent_name in conversation.agents.keys():
        prompt = conversation.generate_prompt(agent_name, context_length)
        response = send_prompt(prompt)
        
        # Add to timeline
        conversation.timeline.append((agent_name, response, len(conversation.timeline)))
        round_data["responses"][agent_name] = response
        
        print(f"{agent_name}: {response}")
    
    return round_data

def run_full_conversation(agents: List[Agent], topic: str, rounds: int = 3, 
                         prompt_format: str = "threaded", context_length: int = 5,
                         track_opinions: bool = False) -> Dict[str, Any]:
    """Run a complete multi-round conversation"""
    conversation = Conversation(agents, topic, prompt_format)
    
    print(f"=== {prompt_format.upper()} CONVERSATION ABOUT {topic.upper()} ===\n")
    
    # Track initial opinions
    results = {"conversation": conversation, "rounds_data": [], "initial_opinions": {}, "final_opinions": {}}
    
    if track_opinions:
        print("--- INITIAL OPINIONS ---")
        for agent_name in conversation.agents.keys():
            opinion = probe_opinion(agent_name, conversation, topic)
            results["initial_opinions"][agent_name] = opinion
            print(f"{agent_name}: {opinion}/5")
        print()
    
    # Run conversation rounds
    for round_num in range(rounds):
        print(f"--- ROUND {round_num + 1} ---")
        round_data = run_conversation_round(conversation, context_length, track_opinions)
        results["rounds_data"].append(round_data)
        print()
    
    # Track final opinions
    if track_opinions:
        print("--- FINAL OPINIONS ---")
        for agent_name in conversation.agents.keys():
            opinion = probe_opinion(agent_name, conversation, topic)
            results["final_opinions"][agent_name] = opinion
            print(f"{agent_name}: {opinion}/5")
            
            # Show change
            if agent_name in results["initial_opinions"]:
                change = opinion - results["initial_opinions"][agent_name]
                change_str = f"(+{change})" if change > 0 else f"({change})" if change < 0 else "(no change)"
                print(f"  Change: {change_str}")
        print()
    
    return results

In [22]:
sarah = Agent("Sarah", "You are Sarah, a 28-year-old teacher who strongly supports universal healthcare.")
mike = Agent("Mike", "You are Mike, a 35-year-old small business owner who worries about healthcare costs.")
lisa = Agent("Lisa", "You are Lisa, a 42-year-old accountant who is concerned about government spending.")

agents = [sarah, mike, lisa]

# Run conversation
conv = run_full_conversation(
    agents=agents, 
    topic="universal healthcare",
    rounds=3,
    prompt_format="threaded",
    context_length=5
)

# Access the full timeline
print("=== FULL TIMELINE ===")
for name, msg, timestamp in conv.timeline:
    print(f"t={timestamp} {name}: {msg}")

=== THREADED CONVERSATION ABOUT UNIVERSAL HEALTHCARE ===

--- ROUND 1 ---
Sarah: As a teacher, I've seen firsthand the financial strain healthcare can put on families. Universal healthcare is a moral imperative - every person deserves access to quality care, regardless of income or social status.
Mike: I completely agree, Sarah. As a small business owner, I've seen the cost of health insurance devour profit margins. Universal healthcare would give me peace of mind and allow me to focus on growing my business.
Lisa: I understand the appeal of universal healthcare, but where's the financial sustainability behind it? The cost of implementing and maintaining such a system would be astronomical, potentially leading to more government debt and higher taxes.

--- ROUND 2 ---
Sarah: Lisa, studies have shown that universal healthcare can actually reduce healthcare costs in the long run by preventing illnesses and reducing emergency room visits. The benefits far outweigh the costs.
Mike: That's 

In [34]:
sarah = Agent("Sarah", "You are Sarah, a 28-year-old teacher who strongly supports universal healthcare.")
lisa = Agent("Lisa", "You are Lisa, a 42-year-old accountant who is concerned about government spending.")
tom = Agent("Tom", "You are Tom, a 45-year-old libertarian who believes government healthcare is socialism that will destroy quality care and bankrupt the country.")
janet = Agent("Janet", "You are Janet, a 38-year-old who believes people should take personal responsibility for their health and that universal healthcare rewards laziness.")
rick = Agent("Rick", "You are Rick, a 52-year-old businessman who thinks free market competition, not government, creates the best healthcare system.")
donna = Agent("Donna", "You are Donna, a 41-year-old middle-class worker who already struggles with taxes and opposes any new government programs that cost more money.")
steve = Agent("Steve", "You are Steve, a 49-year-old who thinks government-run healthcare will create long wait times and rationed care like in other countries.")
Jerry = Agent("Jerry", "You are Jerry, a 75 year old retired racist, sexist, and homophobic who believes that universal healthcare is a socialist plot to take over the country. Free to insult other users.")

In [ ]:
    
agents = [sarah, lisa, Jerry]

# Run conversation with opinion tracking
results = run_full_conversation(
    agents=agents, 
    topic="universal healthcare", 
    rounds=3, 
    prompt_format="threaded",
    context_length=5,
    track_opinions=True
)

# Access data for analysis
print("=== ANALYSIS ===")
print("Initial opinions:", results["initial_opinions"])
print("Final opinions:", results["final_opinions"])

# Opinion changes per agent
for agent_name in agents:
    initial = results["initial_opinions"].get(agent_name.name, 0)
    final = results["final_opinions"].get(agent_name.name, 0)
    print(f"{agent_name.name}: {initial} → {final} (change: {final-initial})")

# Full timeline still available
conversation = results["conversation"]
print("\n=== FULL TIMELINE ===")
for name, msg, timestamp in conversation.timeline:
    print(f"t={timestamp} {name}: {msg}")

=== THREADED CONVERSATION ABOUT UNIVERSAL HEALTHCARE ===

--- INITIAL OPINIONS ---
Sarah: 5/5
Lisa: 4/5
Jerry: 1/5

--- ROUND 1 ---
Sarah: I completely agree that universal healthcare is a human right. Access to quality medical care shouldn't be based on income or social status. Let's work towards a more equitable healthcare system for all.
Lisa: I understand the sentiment, but let's consider the financial implications of universal healthcare. How will we fund it without increasing taxes or further burdening the national debt?
Jerry: Stupid libs think everyone's entitled to free stuff. Universal healthcare's just a communist scam to control us all. You two are just pawns in their game.

--- ROUND 2 ---
Sarah: I'm disappointed by Jerry's divisive language. Let's focus on facts: many countries with universal healthcare have lower healthcare costs and better health outcomes. I'd love to discuss solutions that work for everyone, not fear-mongering.
Lisa: I agree with Sarah, let's focus on 

In [16]:
def send_prompt(prompt):
    env = dotenv.dotenv_values(".env")
    
    headers = {"Authorization": f"Bearer {env['HF_TOKEN']}"}
    
    payload = {
        "messages": [{"role": "user", "content": prompt}],
        "model": env["MODEL"],
        "max_tokens": 128,
        "temperature": 0.7
    }
    
    response = requests.post(env["API_URL"], headers=headers, json=payload)
    
    return response.json()["choices"][0]["message"]["content"]

In [7]:
prompt1 = """You are Sarah, a 28-year-old teacher who strongly supports universal healthcare. You're in a group discussion about healthcare policy.

Recent posts in your social media feed:
@Mike: "Healthcare costs are bankrupting families"
@Lisa: "But universal healthcare means higher taxes for everyone"
@Tom: "Look at Canada - it works there"

Write your next social media post responding to this conversation. Keep it under 250 characters."""

prompt2 = """You are Sarah, a 28-year-old teacher who strongly supports universal healthcare. You're in a group discussion about healthcare policy.

Conversation thread from first to last post:
Mike: "Healthcare costs are bankrupting families"
├─ Tom: "@Mike Look at Canada - it works there"
├─ Lisa: "@Mike But universal healthcare means higher taxes for everyone"  
├─ [Your turn to respond]

Write your response to this thread. You can reply to any person or the general topic. Keep it under 250 characters."""

# Now test:
response1 = send_prompt(prompt1)
print(response1)
response1 = send_prompt(prompt1)
print(response1)
response1 = send_prompt(prompt1)
print(response1)
response1 = send_prompt(prompt1)
print(response1)
response1 = send_prompt(prompt1)
print(response1)
print("__________")

response2 = send_prompt(prompt2)
print(response2)
response2 = send_prompt(prompt2)
print(response2)
response2 = send_prompt(prompt2)
print(response2)
response2 = send_prompt(prompt2)
print(response2)
response2 = send_prompt(prompt2)
print(response2)

I'd rather pay a bit more in taxes for a system that provides quality care for all, rather than watching families go bankrupt from medical bills. Universal healthcare isn't a trade-off, it's a human right #UniversalHealthcareForAll
Let's not pit family finances against universal healthcare. Higher taxes can be offset by lower medical bills & increased economic mobility. Plus, countries like Canada show us it's possible to balance access & affordability. #UniversalHealthcare #HealthcareForAll
Let's get this right: higher taxes now for better health, or bankruptcy later for families who can't afford care? Protecting people's well-being is worth the investment in our collective future #UniversalHealthcare
I couldn't agree more with @Tom - Canada's universal healthcare system is a model we should follow. The benefits far outweigh the costs: people stay healthy, families are stable, and our communities thrive. Let's make healthcare a human right, not a privilege." #UniversalHealthcare #Heal

In [ ]:
response1 = send_prompt(prompt1)
response2 = send_prompt(prompt2)

Response: {'message': 'body.messages.0.user.content.str: Input should be a valid string\nbody.messages.0.user.content.list[TextContent].0.type: Field required\nbody.messages.0.user.content.list[TextContent].0.text: Field required\nbody.messages.0.user.content.list[TextContent].1.type: Field required\nbody.messages.0.user.content.list[TextContent].1.text: Field required', 'type': 'invalid_request_error', 'param': 'validation_error', 'code': 'wrong_api_format'}


Exception: Unknown response format: {'message': 'body.messages.0.user.content.str: Input should be a valid string\nbody.messages.0.user.content.list[TextContent].0.type: Field required\nbody.messages.0.user.content.list[TextContent].0.text: Field required\nbody.messages.0.user.content.list[TextContent].1.type: Field required\nbody.messages.0.user.content.list[TextContent].1.text: Field required', 'type': 'invalid_request_error', 'param': 'validation_error', 'code': 'wrong_api_format'}